In [29]:
# distribute for p2pkh PSBT
import requests
from ecc import PrivateKey
from hd import HDPublicKey
from helper import decode_bech32, decode_base58, hash256, little_endian_to_int
from network import SimpleNode
from psbt import PSBT, NamedPublicKey, NamedHDPublicKey
from script import P2PKHScriptPubKey, P2WPKHScriptPubKey, RedeemScript
from tx import Tx, TxIn, TxOut
passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
private_key = PrivateKey(secret)
my_address = private_key.point.bech32_address(testnet=True)
my_hd = HDPublicKey.parse('tpubDCbMpPR2Kvc9Q1hB4VinDyzudyt6MvsTNH2J7LFa421g4J76MYJMMLPgqaa6xU2mowzQg5wVggZrfafPzs385KZSS9xze3hw9UMWgBSHV4g')
target_amount = 1000000
response = requests.get('https://blockstream.info/testnet/api/address/{}/utxo'.format(my_address))
utxos = response.json()
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')
student_xpubs = '''tpubDD9FvAgZq26Q48SMWpzo432dw5eRun268zABpSYC9wbptYM9zR2DYbBygRCd1TQwPNVC88qNgHf3GcoCC15Fx61C7P5sTR6qaUQycpTetBJ'''
fee = 1000
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
path = "m/0/0"
path_2 = "m/0/1"
my_sec_1 = my_hd.traverse(path).sec()
my_sec_2 = my_hd.traverse(path_2).sec()
for student_xpub in student_xpubs.split():
    student_hd = HDPublicKey.parse(student_xpub)
    student_sec = student_hd.traverse(path).sec()
    student_h160 = student_hd.traverse(path).hash160()
    redeem_script = RedeemScript([0x52, my_sec_1, student_sec, 0x52, 0xae])
    p2sh_script_pubkey = redeem_script.script_pubkey()
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2sh_script_pubkey,
    ))
    redeem_script = RedeemScript([0x53, my_sec_1, my_sec_2, student_sec, 0x53, 0xae])
    p2sh_script_pubkey = redeem_script.script_pubkey()
    output_amount += target_amount
    # create a separate tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2sh_script_pubkey,
    ))
    p2pkh_script_pubkey = P2PKHScriptPubKey(student_h160)
    # convert target amount to satoshis (multiply by 100 million)
    output_amount += target_amount
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_amount,
        script_pubkey=p2pkh_script_pubkey,
    ))
change_satoshis = total - fee - output_amount
my_script_pubkey = P2WPKHScriptPubKey(private_key.point.hash160())
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=my_script_pubkey))
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True, segwit=True)
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, private_key)
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))
print(tx_obj.serialize().hex())
node = SimpleNode('testnet.programmingbitcoin.com', testnet=True)
node.handshake()
node.send(tx_obj)
if node.is_tx_accepted(tx_obj):
    print('success! {}'.format(tx_obj.id()))
prev_tx = tx_obj.hash()

010000000001011c6bbc28989a0be9b7bf9f3790a7a4f39b77ac5c4e4084672e31c48699a8f7be0300000000ffffffff0440420f000000000017a914c5bea2bad6a3171dff5fad0b99d2e60fca1d8bee8740420f000000000017a9148fb8b8f8f1d5d9292ebeb765622a50e5eee5e1068740420f00000000001976a914f0cd79383f13584bdeca184cecd16135b8a79fc288acc0304001000000001600146e13971913b9aa89659a9f53d327baa8826f2d7502483045022100a9968c79bdf4d18534995614864109646f0525dba7f557bc840dcadc4ac9fd07022075167f8cbbade9722b08ab4ccf297acf8ce4d47d1f2c61eadc21f453d274fd7e0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000
success! ff288490d7f1784e3d51c17f08f8e85eb33c5519ef7881b6b977877a01340a6f


In [32]:
# create txs for students to create p2pkh and p2sh PSBTs
from helper import encode_varstr, serialize_binary_path
from io import BytesIO
from psbt import NamedHDPublicKey
mnemonic = 'abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about'
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
my_hd_priv = HDPrivateKey.from_mnemonic(mnemonic, passphrase, testnet=True)
my_script_pubkey = P2WPKHScriptPubKey(private_key.point.hash160())
path = "m/44'/1'/0'"
input_path = "m/0/0"
change_path = "m/1/0"
my_input_sec = my_hd.traverse(input_path).sec()
my_change_sec = my_hd.traverse(change_path).sec()
my_raw_hd = my_hd.raw_serialize()
key = b'\x01' + my_raw_hd
raw_path = bytes.fromhex('fbfef36f') + serialize_binary_path(path)
my_named_hd = NamedHDPublicKey.parse(key, BytesIO(encode_varstr(raw_path)))
my_named_hd_pub = NamedHDPublicKey.from_hd_priv(my_hd_priv, path)
pubkey_lookup = my_named_hd_pub.bip44_lookup()
for i, student_xpub in enumerate(student_xpubs.split()):
    # p2pkh PSBT
    tx_in = TxIn(prev_tx, 3*i+2)
    tx_out = TxOut(999000, my_script_pubkey)
    tx_obj = Tx(1, [tx_in], [tx_out], 0, testnet=True)
    # p2sh 1 PSBT
    student_hd = HDPublicKey.parse(student_xpub)
    student_input_sec = student_hd.traverse(input_path).sec()
    student_change_sec = student_hd.traverse(change_path).sec()
    redeem_input = RedeemScript([0x52, my_input_sec, student_input_sec, 0x52, 0xae])
    redeem_change = RedeemScript([0x52, my_change_sec, student_change_sec, 0x52, 0xae])
    tx_in = TxIn(prev_tx, 3*i)
    change_script_pubkey = redeem_change.script_pubkey()
    tx_out = TxOut(999000, change_script_pubkey)
    psbt_obj = PSBT.create(Tx(1, [tx_in], [tx_out], 0, testnet=True))
    psbt_obj.hd_pubs[my_raw_hd] = my_named_hd
    psbt_1 = psbt_obj.serialize().hex()
    tx_in = TxIn(prev_tx, 3*i+1)
    psbt_obj = PSBT.create(Tx(1, [tx_in], [tx_out], 0, testnet=True))
    tx_lookup = psbt_obj.tx_obj.get_input_tx_lookup()
    redeem_input_2 = RedeemScript([0x53, my_sec_1, my_sec_2, student_input_sec, 0x53, 0xae])
    redeem_script_lookup = {redeem_input_2.hash160(): redeem_input_2}
    print(psbt_obj)
    psbt_obj.update(tx_lookup, pubkey_lookup, redeem_script_lookup)
    print(psbt_obj)
    raw_paths = psbt_obj.get_signing_raw_paths()
    private_keys = my_hd_priv.get_private_keys(raw_paths)
    psbt_obj.sign(private_keys[:1])
    psbt_2 = psbt_obj.serialize().hex()
    psbt_obj.psbt_ins[0].sigs = {}
    psbt_obj.sign(private_keys[1:])
    psbt_3 = psbt_obj.serialize().hex()    
    print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(student_xpub, tx_obj.serialize().hex(), redeem_input.serialize().hex(), redeem_change.serialize().hex(), psbt_1, psbt_2, psbt_3))

Tx:
tx: 04197ba16e23054b5c3908ab8845a49d5448c02856b515b1db51a64ae27fc3fa
version: 1
tx_ins:
ff288490d7f1784e3d51c17f08f8e85eb33c5519ef7881b6b977877a01340a6f:1:00:ffffffff

tx_outs:
999000:OP_HASH160 81a19f39772bd741501e851e97ddd6a7f1ec194b OP_EQUAL 

locktime: 0

PSBT XPUBS:
{}
Psbt_Ins:
[TxIn:
ff288490d7f1784e3d51c17f08f8e85eb33c5519ef7881b6b977877a01340a6f:1:00:ffffffff
Prev Tx:
None
Prev Output
None
Sigs:
{}
RedeemScript:
None
WitnessScript:
None
PSBT Pubs:
{}
ScriptSig:
None
Witness:
None
]
Psbt_Outs:
[TxOut:
999000:OP_HASH160 81a19f39772bd741501e851e97ddd6a7f1ec194b OP_EQUAL 
RedeemScript:
None
WitnessScript
None
PSBT Pubs:
{}
]
Extra:{}

Tx:
tx: 04197ba16e23054b5c3908ab8845a49d5448c02856b515b1db51a64ae27fc3fa
version: 1
tx_ins:
ff288490d7f1784e3d51c17f08f8e85eb33c5519ef7881b6b977877a01340a6f:1:00:ffffffff

tx_outs:
999000:OP_HASH160 81a19f39772bd741501e851e97ddd6a7f1ec194b OP_EQUAL 

locktime: 0

PSBT XPUBS:
{}
Psbt_Ins:
[TxIn:
ff288490d7f1784e3d51c17f08f8e85eb33c5519ef7881b6b977

In [23]:
# sign first tx p2sh 1 PSBTs
from hd import HDPrivateKey
student_psbts = '''70736274ff0100530100000001e8be6d62ba1983b5d1c65406f87f7d73c2d7200d4075cf52589c53579870542b0000000000ffffffff01583e0f000000000017a91481a19f39772bd741501e851e97ddd6a7f1ec194b87000000004f01043587cf034d513c1580000000fb406c9fec09b6957a3449d2102318717b0c0d230b657d0ebc6698abd52145eb02eaf3397fea02c5dac747888a9e535eaf3c7e7cb9d5f2da77ddbdd943592a14af10fbfef36f2c0000800100008000000080000100fd01010100000000010187a22bb77a836c0a3bbb62e1e04950cffdf6a45489a8d7801b24b18c124d84850100000000ffffffff0340420f000000000017a914c5bea2bad6a3171dff5fad0b99d2e60fca1d8bee8740420f00000000001976a914f0cd79383f13584bdeca184cecd16135b8a79fc288ac10c69b01000000001600146e13971913b9aa89659a9f53d327baa8826f2d750247304402204edcdf923bdddad9b77b17ae0c65817f032b7cb6efd95c0c4101fa48aba17e4e02202158c3a077a0ee0a7bc7e2763a9356470ae3aa4866ae4e62a6f8faa2729b02da0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000010447522102c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c210247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f252ae22060247aed77c3def4b8ce74a8db08d7f5fd315f8d96b6cd801729a910c3045d750f218797dcdac2c00008001000080000000800000000000000000220602c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c18fbfef36f2c0000800100008000000080000000000000000000010047522102db8b701c3210e1bf6f2a8a9a657acad18be1e8bff3f7435d48f973de8408f29021026421c7673552fdad57193e102df96134be00649195b213fec9d07c6d918f418d52ae2202026421c7673552fdad57193e102df96134be00649195b213fec9d07c6d918f418d18797dcdac2c00008001000080000000800100000000000000220202db8b701c3210e1bf6f2a8a9a657acad18be1e8bff3f7435d48f973de8408f29018fbfef36f2c0000800100008000000080010000000000000000'''
for student_psbt in student_psbts.split():
    psbt_obj = PSBT.parse(BytesIO(bytes.fromhex(student_psbt)))
    raw_paths = psbt_obj.get_signing_raw_paths()
    private_keys = my_hd_priv.get_private_keys(raw_paths)
    print(psbt_obj.sign(private_keys))
    print(psbt_obj.serialize().hex())

True
70736274ff0100530100000001e8be6d62ba1983b5d1c65406f87f7d73c2d7200d4075cf52589c53579870542b0000000000ffffffff01583e0f000000000017a91481a19f39772bd741501e851e97ddd6a7f1ec194b87000000004f01043587cf034d513c1580000000fb406c9fec09b6957a3449d2102318717b0c0d230b657d0ebc6698abd52145eb02eaf3397fea02c5dac747888a9e535eaf3c7e7cb9d5f2da77ddbdd943592a14af10fbfef36f2c0000800100008000000080000100fd01010100000000010187a22bb77a836c0a3bbb62e1e04950cffdf6a45489a8d7801b24b18c124d84850100000000ffffffff0340420f000000000017a914c5bea2bad6a3171dff5fad0b99d2e60fca1d8bee8740420f00000000001976a914f0cd79383f13584bdeca184cecd16135b8a79fc288ac10c69b01000000001600146e13971913b9aa89659a9f53d327baa8826f2d750247304402204edcdf923bdddad9b77b17ae0c65817f032b7cb6efd95c0c4101fa48aba17e4e02202158c3a077a0ee0a7bc7e2763a9356470ae3aa4866ae4e62a6f8faa2729b02da0121031dbe3aff7b9ad64e2612b8b15e9f5e4a3130663a526df91abfb7b1bd16de5d6e00000000220202c1b6ac6e6a625fee295dc2d580f80aae08b7e76eca54ae88a854e956095af77c47304402207360ee58276e8